In [1]:
# Model Experimentation
# Only run once
import sys
import os

project_root = os.path.abspath('../')
os.chdir(project_root)

src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Setup

In [13]:
from load_data import DataLoader
from preprocess import Preprocessor, FeatureEngineering
from evaluate_model import Evaluator
from model_definition import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import joblib
import lightgbm as lgb
import optuna
import mlflow
import warnings
warnings.filterwarnings("ignore")

dl = DataLoader()
pre = Preprocessor()
fe = FeatureEngineering()
evaluator = Evaluator()

In [ ]:
mlflow.set_tracking_uri("file:///C:/Users/Axioo/Documents/Fahmi/ai/ml/predicting-loan-payback/logs")

In [4]:
train = dl.load_data('train_clean.csv', 'data/processed')
test = dl.load_data('test_clean.csv', 'data/processed')
submission = dl.load_data('sample_submission.csv', 'data/raw')

train_df = train.copy()
test_df = test.copy()

# Feature engineering

In [5]:
# Add feature

def loan_to_income_ratio(df):
    df['loan_to_income_ratio'] = df['loan_amount'] / (df['annual_income'] + 1e-6)
    df = fe.log_transform(df, 'loan_to_income_ratio')
    return df

def monthly_debt(df):
    df['monthly_debt'] = df['annual_income'] * df['debt_to_income_ratio'] / 12
    return df

def payment_to_income(df):
    monthly_rate =  df['interest_rate'] / 100 / 12
    compound_factor = (1 + monthly_rate) ** 360
    numerator = df['loan_amount'] * monthly_rate * compound_factor
    denominator = compound_factor - 1
    df['payment_to_income'] = numerator / denominator
    return df

def subgrade_numeric(df):
    grade_map = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6}
    df['subgrade_numeric'] = (
        df['grade_subgrade'].str[0].map(grade_map) * 5 + 
        df['grade_subgrade'].str[1].astype(int)
    )
    return df

def interest_per_grade(df):
    df['interest_per_grade'] = df['interest_rate'] / df['subgrade_numeric']
    return df

def credit_score_bins(df):
    df = fe.binning(df, 'credit_score', bins=[-np.inf, 579, 669, 739, 799, 850, np.inf], labels=['Very_Poor', 'Poor', 'Fair', 'Good', 'Excellent', 'Exceptional'], include_lowest=True)
    bins = [-np.inf, 579, 669, 739, 799, 850, np.inf]
    df['credit_score_bin_num'] = pd.cut(df['credit_score'], bins=bins, labels=range(1,7), include_lowest=True)
    df['credit_score_bin_num'] = df['credit_score_bin_num'].astype(int)
    return df

def get_expected_score(df):
    return df.groupby('grade_subgrade')['credit_score'].mean().to_dict()

def grade_diff(df, expected_score_dict):
    df['expected_score'] = df['grade_subgrade'].map(expected_score_dict)
    median_diff = (df['credit_score'] - df['expected_score']).median()
    df['score_grade_diff'] = df['credit_score'] - df['expected_score']
    df['score_grade_diff'] = df['score_grade_diff'].fillna(median_diff)
    return df

def log_annual_income(df):
    df['annual_income_log'] = df['annual_income']
    df = fe.log_transform(df, 'annual_income_log')
    return df

def log_loan_amount(df):
    df['loan_amount_log'] = df['loan_amount']
    df = fe.log_transform(df, 'loan_amount_log')
    return df

In [ ]:
train_df = loan_to_income_ratio(train_df)
test_df = loan_to_income_ratio(test_df)

train_df = monthly_debt(train_df)
test_df = monthly_debt(test_df)

train_df = payment_to_income(train_df)
test_df = payment_to_income(test_df)

expected_scores = get_expected_score(train_df)

train_df = subgrade_numeric(train_df)
train_df = interest_per_grade(train_df)
train_df = credit_score_bins(train_df)
train_df = grade_diff(train_df, expected_scores)

test_df = subgrade_numeric(test_df)
test_df = interest_per_grade(test_df)
test_df = credit_score_bins(test_df)
test_df = grade_diff(test_df, expected_scores)

train_df = log_annual_income(train_df)
test_df = log_annual_income(test_df)

train_df = log_loan_amount(train_df)
test_df = log_loan_amount(test_df)

# Tuning hyperparameter

In [14]:
X = train_df.drop(columns=['id', 'loan_paid_back'])
y = train_df['loan_paid_back']

X_test = test_df.drop(columns=['id'])

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

for col in categorical_cols:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype('category')
        X_val[col]   = X_val[col].astype('category')

def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "random_state": 42,
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 50, 300),
        "max_depth": trial.suggest_int("max_depth", -1, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "min_child_weight": trial.suggest_float("min_child_weight", 1e-3, 10, log=True),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-3, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-3, 10.0, log=True),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.0, 1.0),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 5.0),
    }
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)
    
    with mlflow.start_run(nested=True):
        model = lgb.train(
            params,
            dtrain,
            valid_sets=[dval],
            num_boost_round=500,
            callbacks=[lgb.early_stopping(100)]
        )

        preds = model.predict(X_val, num_iteration=model.best_iteration)
        auc = roc_auc_score(y_val, preds)

        mlflow.log_params(params)
        mlflow.log_metric("val_auc", auc)
        mlflow.log_metric("best_iteration", model.best_iteration)
        mlflow.log_metric("train_auc", roc_auc_score(y_train, model.predict(X_train, num_iteration=model.best_iteration)))
        
        signature = mlflow.models.infer_signature(X_train, preds)
        mlflow.lightgbm.log_model(
            model, 
            name="model",
            signature=signature,
            input_example=None
        )

    return auc

In [18]:
mlflow.set_experiment("lgbm")
with mlflow.start_run(run_name="lgbm_hyperparameter_tuning_1"):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)
    
    best_params = study.best_params
    best_value = study.best_value
    
    print(f"Best AUC: {study.best_value:.6f}")
    print("Best params:")
    for k, v in study.best_params.items():
        print(f"  {k}: {v}")
    
    mlflow.log_params(best_params)
    mlflow.log_metric("best_val_auc", study.best_value)
    mlflow.log_metric("n_trials_completed", len(study.trials))

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's auc: 0.92103


2025/11/22 23:36:20 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
ValueError('train and valid dataset categorical_feature do not match.')Traceback (most recent call last):
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\utils\_capture_modules.py", line 166, in load_model_and_predict
    model.predict(input_example, params=params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\lightgbm\__init__.py", line 511, in predict
    return self.lgb_model.predict(dataframe)
           ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py", line 4767, in predict
    return predictor.predict(
           ~~~~~~~~~~~~~~~~~^
        data=data,
        ^^^^^^^^^^
    ...<6 lines>...
        validate_features=val

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[417]	valid_0's auc: 0.920311


2025/11/22 23:36:41 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
ValueError('train and valid dataset categorical_feature do not match.')Traceback (most recent call last):
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\utils\_capture_modules.py", line 166, in load_model_and_predict
    model.predict(input_example, params=params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\lightgbm\__init__.py", line 511, in predict
    return self.lgb_model.predict(dataframe)
           ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py", line 4767, in predict
    return predictor.predict(
           ~~~~~~~~~~~~~~~~~^
        data=data,
        ^^^^^^^^^^
    ...<6 lines>...
        validate_features=val

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[455]	valid_0's auc: 0.920032


...

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[480]	valid_0's auc: 0.920771


2025/11/22 23:53:18 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
ValueError('train and valid dataset categorical_feature do not match.')Traceback (most recent call last):
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\utils\_capture_modules.py", line 166, in load_model_and_predict
    model.predict(input_example, params=params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\lightgbm\__init__.py", line 511, in predict
    return self.lgb_model.predict(dataframe)
           ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py", line 4767, in predict
    return predictor.predict(
           ~~~~~~~~~~~~~~~~~^
        data=data,
        ^^^^^^^^^^
    ...<6 lines>...
        validate_features=val

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[460]	valid_0's auc: 0.921318


2025/11/22 23:53:36 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
ValueError('train and valid dataset categorical_feature do not match.')Traceback (most recent call last):
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\utils\_capture_modules.py", line 166, in load_model_and_predict
    model.predict(input_example, params=params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlflow\lightgbm\__init__.py", line 511, in predict
    return self.lgb_model.predict(dataframe)
           ~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
  File "c:\Users\Axioo\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py", line 4767, in predict
    return predictor.predict(
           ~~~~~~~~~~~~~~~~~^
        data=data,
        ^^^^^^^^^^
    ...<6 lines>...
        validate_features=val

Best AUC: 0.921744
Best params:
  learning_rate: 0.07506481389351985
  num_leaves: 257
  max_depth: 9
  min_child_samples: 86
  min_child_weight: 0.001135919445521494
  feature_fraction: 0.6444563658740958
  bagging_fraction: 0.9256663942037113
  bagging_freq: 4
  lambda_l1: 3.8546178890730762
  lambda_l2: 0.0021516695834795114
  min_gain_to_split: 0.7798854694703099
  scale_pos_weight: 1.464428475044859


# Model

In [20]:
final_params = best_params.copy()
final_params.update({
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "random_state": 42,
    "verbosity": -1,
})

for col in categorical_cols:
    if col in X.columns:
        X[col] = X[col].astype('category')

dtrain_full = lgb.Dataset(X, label=y)
final_model = lgb.train(
    final_params,
    dtrain_full,
    num_boost_round=1000,
)

joblib.dump(final_model, "lgbm_model.pkl")

['lgbm_model.pkl']

# Predict

In [23]:
expected_score_dict = {'A1': 816.643125, 'A2': 816.3979187314172, 'A3': 816.2647628267183, 'A4': 818.1263962375074, 'A5': 816.1974908943747, 'B1': 759.0757808142778, 'B2': 758.985758554757, 'B3': 758.9071520896165, 'B4': 760.0682424155077, 'B5': 760.3436177082586, 'C1': 702.838802166295, 'C2': 701.498521389343, 'C3': 701.0708407871199, 'C4': 701.6006040352413, 'C5': 701.4685184837857, 'D1': 644.6651003267709, 'D2': 644.4268413104089, 'D3': 645.2013953234862, 'D4': 644.8396444140525, 'D5': 644.892495560886, 'E1': 594.0909882455377, 'E2': 593.7115505335844, 'E3': 594.3012014134275, 'E4': 594.6345196615232, 'E5': 593.5843195266273, 'F1': 557.9859053126129, 'F2': 558.2523544109167, 'F3': 559.5975993703266, 'F4': 557.910569105691, 'F5': 559.4968891878258}

In [ ]:
test_predict = dl.load_data('test_final.csv', 'data/processed')

test_predict = loan_to_income_ratio(test_predict)
test_predict = monthly_debt(test_predict)
test_predict = payment_to_income(test_predict)
test_predict = subgrade_numeric(test_predict)
test_predict = interest_per_grade(test_predict)
test_predict = credit_score_bins(test_predict)
test_predict = grade_diff(test_predict, expected_score_dict)
test_predict = log_annual_income(test_predict)
test_predict = log_loan_amount(test_predict)

categorical_cols = test_predict.select_dtypes(include=['object', 'category']).columns.tolist()

for col in categorical_cols:
    if col in test_predict.columns:
        test_predict[col] = test_predict[col].astype('category')

In [35]:
test_predict = test_predict[X.columns]

model = joblib.load('models/lgbm_model.pkl')
pred = model.predict(test_predict)
pred = np.clip(pred, 0, 1)

submission = dl.load_data('sample_submission.csv', 'data/raw')
target_col = [col for col in submission.columns if col not in ['id', 'Id', 'ID']][0]
submission[target_col] = pred
submission.to_csv('output/submission.csv', index=False)
print(f"Prediksi disimpan ke kolom: {target_col}")

Prediksi disimpan ke kolom: loan_paid_back
